In [50]:
#import sys
#sys.executable
#sys.path

In [64]:
from bokeh.plotting import figure, show
#from bokeh.plotting import output_file
from bokeh.plotting import output_notebook
import numpy as np
from bokeh.io import output_file

import getpass
name = getpass.getuser()
import datetime

now=    datetime.datetime.now().strftime('-%Y%m%d-%H:%M:%S')
output_file('teste_'+name+now, title='Bokeh Plot')
print(now)



-20170929-14:33:32


In [71]:
%matplotlib inline
import yaml

def name_file(ubr):
    folder = '/home/foliveira/quicklookOLD/spectro/redux/exposures/20190101/00000003/'
    file_filter =  'ql-snr-'+ubr+'-00000003.yaml'
    print('Loading ',file_filter)
    return folder + file_filter

keys    = []
docs    = []
metrics = []
for i, iname in enumerate(['r3']):
    print(iname)
    stream = open( name_file(iname), "r")
    docs.append( yaml.load(stream) ) 
    keys.append( list(docs[i]['METRICS'].keys()) )
    metrics.append(docs[i]['METRICS'])
    #print( metrics[-1]['ELG_FITRESULTS']),'\n'

print('\n\nAvailable data and Keys:')    
for i, k in enumerate(keys[0]):
    print(i,':',k)
#print(keys,'\n\n', metrics[0]['ELG_SNR_MAG'][1]) 
#print metrics[0]['ELG_SNR_MAG'][0][0:3],'\n',metrics[1]['ELG_SNR_MAG'][0][0:3]

r3
Loading  ql-snr-r3-00000003.yaml


Available data and Keys:
0 : QSO_SNR_MAG
1 : LRG_FIBERID
2 : STAR_FITRESULTS
3 : FIDSNR_WARN
4 : LRG_FIDMAG_SNR
5 : LRG_SNR_MAG
6 : ELG_FIBERID
7 : STAR_FIDMAG_SNR
8 : ELG_FIDMAG_SNR
9 : QSO_FITRESULTS
10 : ELG_SNR_MAG
11 : STAR_SNR_MAG
12 : RA
13 : LRG_FITRESULTS
14 : QSO_FIBERID
15 : NUM_NEGATIVE_SNR
16 : DEC
17 : QSO_FIDMAG_SNR
18 : MEDIAN_SNR
19 : ELG_FITRESULTS
20 : STAR_FIBERID


In [78]:
from bokeh.models import HoverTool
from bokeh.plotting import ColumnDataSource

p = figure(plot_width=400, plot_height=400)

i_snr = 10 # Which Object is being analysed (check keys list above)

color= ['red', 'blue', 'green']
from bokeh.models.glyphs import Circle, Line

for ii,iname in enumerate(['r0']):#(['r0','b0']):
    x = metrics[ii][keys[ii][i_snr]][1]
    y = metrics[ii][keys[ii][i_snr]][0]

    source = ColumnDataSource(data=dict(
    x1=x,
    y1=y,
    ))
    
    hover = HoverTool(tooltips=[
         ("data_index", "$index"),
         ("(Mag,SNR)", "($x, $y)")
         ])
    p =  figure(tools = [hover,'pan','wheel_zoom','box_zoom','reset'])
    
    p.xaxis.axis_label = "DECAM"
    p.yaxis.axis_label = "SNR"
    p.title.text = keys[ii][i_snr].replace('_SNR_MAG','')

    a, b       = np.polyfit(x,y,1) # Ajuste Linear
    a2, b2, c2 = np.polyfit(x,y,2) # Ajuste de 2a ordem

    xfit = np.linspace(np.min(x), np.max(x),200)
    fit_lin = ColumnDataSource(data = dict( x = xfit, y = a*xfit+ b)  )
    fit_sec = ColumnDataSource(data = dict( x = xfit, y = a2*xfit**2 + b2 *xfit +c2)  )

    # Plots
    p.circle('x1', 'y1', source =source, fill_alpha=0.7, size=7, line_color='black', fill_color=color[ii])         
    p.line(   'x', 'y', source=fit_lin, line_color="black", line_width= 3, alpha=1.)#color[ii],
    p.line(   'x', 'y', source=fit_sec, line_color="black", line_dash="10 10", line_width=3, alpha=1.) #color[ii],   

    print ('\n\n== {} ===:\n1st:\ny = {:3.2e} + {:3.2e} *x \n2nd:\ny = {:3.2e} + {:3.2e} *x + {:3.2e} *x**2\n\n'.format(iname,b,a,c2,b2,a2))

    print ('From pipeline, {}\n[{}, {}, {} ]'.format(keys[0][i_snr].replace('_SNR_MAG',''),c2,b2,a2)) 



== r0 ===:
1st:
y = 1.67e+01 + -7.12e-01 *x 
2nd:
y = 1.64e+02 + -1.40e+01 *x + 2.99e-01 *x**2


From pipeline, ELG
[164.08843628842524, -13.993302419627337, 0.29895178000866135 ]


In [79]:
show(p)